In [1]:
from langchain.document_loaders import PyPDFLoader, TextLoader, CSVLoader, UnstructuredWordDocumentLoader, UnstructuredPowerPointLoader,WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

d:\Anaconda\envs\Basic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


1.2 文本分隔

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

loader = TextLoader('a.txt', encoding='utf8')
document = loader.load()
data = text_splitter.split_documents(document)
data

[Document(metadata={'source': 'a.txt'}, page_content='You have a dog.\nYou have a pig.\n\nSo I want to have one.')]

1.3 向量嵌入

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
import numpy as np

# 初始化嵌入模型
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 生成嵌入向量
texts = [doc.page_content for doc in split_documents]
text_embeddings = embeddings_model.embed_documents(texts)

# 嵌入向量是浮点数列表
print(f"每个文本的嵌入维度: {len(text_embeddings[0])}")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3892\1065644366.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


NameError: name 'split_documents' is not defined

In [ ]:
# 4、向量存储
from langchain.vectorstores import Chroma

Chroma.from_documents(
    data, 
    embeddings_model,
    persist_directory="./ "
).persist()

In [ ]:
# 1、查询嵌入
def get_query_embedding(query, embeddings_model):
    """将查询转换为嵌入向量"""
    return embeddings_model.embed_query(query)

# 示例
user_query = "机器学习的主要应用领域有哪些？"
query_embedding = get_query_embedding(user_query, embeddings_model)

In [ ]:
def retrieve_relevant_documents(query, vectorstore, top_k=5, strategy="similarity"):
    """检索相关文档"""
    if strategy == "similarity":
        # 相似度搜索(正、余弦相似度)
        results = vectorstore.similarity_search(query, k=top_k)
    elif strategy == "mmr":
        # 最大边际相关性搜索（平衡相关性和多样性）
        results = vectorstore.max_marginal_relevance_search(query, k=top_k)
    
    return results

# 检索相关文档
relevant_docs = retrieve_relevant_documents(user_query, vectorstore, top_k=3)

NameError: name 'vectorstore' is not defined